<a href="https://colab.research.google.com/github/Zonas89/lab-3-2/blob/master/Mini_Project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install hl7
!pip install fhir.resources

     |████████████████████████████████| 2.3MB 5.6MB/s 
     |████████████████████████████████| 10.1MB 21.1MB/s 
     |████████████████████████████████| 245kB 35.0MB/s 


In [ ]:
# Make sure you installed package hl7 and fhir.resources
import hl7
from fhir.resources.patient import Patient
from fhir.resources.observation import Observation 
import json
# Message is made based on HL7V2.4
message = 'MSH|^~\&|GHH LAB|ELAB-3|GHH OE|BLDG4|200202150930||ORU^R01|CNTRL-3456|P|2.4\r'
message += 'PID|||555-44-4444||EVERYWOMAN^EVE^E^^^^L|JONES|19620320|F|||153 FERNWOOD DR.^^STATESVILLE^OH^35292||(206)3345232|(206)752-121||||AC555444444||67-A4335^OH^20030520\r'
message += 'OBR|1|845439^GHH OE|1045813^GHH LAB|1554-5^GLUCOSE|||200202150730||||||||555-55-5555^PRIMARY^PATRICIA P^^^^MD^^LEVEL SEVEN HEALTHCARE, INC.|||||||||F||||||444-44-4444^HIPPOCRATES^HOWARD H^^^^MD\r'
message += 'OBX|1|SN|1554-5^GLUCOSE^POST 12H CFST:MCNC:PT:SER/PLAS:QN||^182|mg/dl|70_105|H|||F\r'

h = hl7.parse(message)

# Parse Patient
PID = h['PID']
#Id
PatientId = str(PID[0][3])
#Name
Name = PID[0][5].__str__().split("^")
GivenName = Name[1] +" "+ Name[2]
FamilyName = Name[0]
FullName = GivenName+". "+FamilyName
#BirthDate
Birthdate1 = hl7.parse_datetime(h['PID.7'])
Birthdate = str(Birthdate1.date())
#Gender
dictsex = {'F':'Female', 'M':'Male', 'A':'Unknown','O':'Other','N':'Other','U':'Unknown'}
Sex =  dictsex[str(PID[0][8])]
#Address
Address = PID[0][11].__str__().split("^")
Address_line = Address[0]
Address_city = Address[2]
Address_state = Address[3]
Address_postalCode = Address[4]
#telecom
Telecom_home = str(PID[0][13])
Telecom_work = str(PID[0][14])
#Driver License
DL = PID[0][20].__str__().split("^")
DL_number =str(DL[0])
DL_issue_state = str(DL[1])
DL_expiration_datetime = hl7.parse_datetime(DL[2])
DL_expiration_date = str(DL_expiration_datetime.date())

In [ ]:
print(DL_expiration_date)

2003-05-20


In [ ]:
# Construct Patient in dict
Patient_String = {
  "resourceType": "Patient",
  "id":PatientId,
  "identifier":[{
      "use":"official", 
      "type":{"text":"Driver License"}, 
      "value":DL_number
      }],
  "name":[{
      'family':FamilyName, 
      'given':[GivenName], 
      'text':FullName
      }],
  "gender":Sex,
  "birthDate":Birthdate,
  "address":[{
      "line":[Address_line],
      "city":Address_city,
      "state":Address_state,
      "postalCode":Address_postalCode
      }],
  "telecom":[{
      "system":"phone",
      "value":Telecom_home,
      "use":"home"
        },
      {"system":"phone",
       "value":Telecom_work,
       "use":"work"
       }],
}
print(Patient_String)
# Construct Patient Object
pat = Patient.parse_obj(Patient_String)


{'resourceType': 'Patient', 'id': '555-44-4444', 'identifier': [{'use': 'official', 'type': {'text': 'Driver License'}, 'value': '67-A4335'}], 'name': [{'family': 'EVERYWOMAN', 'given': ['EVE E'], 'text': 'EVE E. EVERYWOMAN'}], 'gender': 'Female', 'birthDate': '1962-03-20', 'address': [{'line': ['153 FERNWOOD DR.'], 'city': 'STATESVILLE', 'state': 'OH', 'postalCode': '35292'}], 'telecom': [{'system': 'phone', 'value': '(206)3345232', 'use': 'home'}, {'system': 'phone', 'value': '(206)752-121', 'use': 'work'}]}


In [ ]:
#Use fhir.resources to validate the Patient object
print(isinstance(pat, Patient))

True


In [ ]:
#Parse Observation
OBR = h['OBR']
OBX = h['OBX']
# Observation Id, using OBR-3 + OBX-3 per Observation mapping (OBX-4 is not applicable)
ObservationId = OBR[0][3].__str__().split("^")[0]+OBX[0][3].__str__().split("^")[0]
ObservationCodeSystem = "http://loinc.org"
ObservationCodeValue = OBX[0][3].__str__().split("^")[0]
ObservationCodeDisplay = OBX[0][3].__str__().split("^")[1] +" "+OBX[0][3].__str__().split("^")[2]
#Subject
S_reference_value = "Patient/"+str(PatientId)
S_display = GivenName+". "+FamilyName
#Interpretation
I_coding_system = "http://terminology.hl7.org/ValueSet/v3-ObservationInterpretationNormality" #V2 url is out of date
I_coding_value = str(OBX[0][8])
Interpretationdict = {'A':'Abnormal', 'AA':'Critical abnormal', 'HH':'Critical High','LL':'Critical Low','H':'High','HU':'Significantly high','L':'Low','LU':'Significant low','N':'Normal'} # Mapping valueset per CodeSystem URL above
I_display = Interpretationdict[str(OBX[0][8])]
#IssuedValue
Issued_value = hl7.parse_datetime(str(OBR[0][7]))
# ValueQuantity 
VQ_Value = OBX[0][5].__str__().split("^")[1]
VQ_units = str(OBX[0][6])
VQ_system = "http://unitsofmeasure.org"
# Performer
P_reference_value = "Practitioner/"+OBR[0][30].__str__().split("^")[0]
P_reference_display = OBR[0][30].__str__().split("^")[2] + ". " + OBR[0][30].__str__().split("^")[1]
# ReferenceRange
Low_value = OBX[0][7].__str__().split("_")[0]
High_value = OBX[0][7].__str__().split("_")[1]
RR_unit = str(OBX[0][6])
RR_system = "http://unitsofmeasure.org"
RR_Code_value = str(OBX[0][6])


In [ ]:
Observation_String ={
    "resourceType": "Observation",
    "id":ObservationId,
    "identifier":[{"use":"official", "value": ObservationId}],
    "status":"final",
    "code":{"coding":[{
      "system": ObservationCodeSystem,
      "code": ObservationCodeValue,
      "display": ObservationCodeDisplay
    }]},    
    "subject":{
        "reference": S_reference_value,
        "display": S_display
    },
    "interpretation":[
      {
          "coding":[
                    {
                        "system": I_coding_system,
                        "code": I_coding_value,
                        "display": I_display
                    }
          ]
      }                
    ],
    "valueQuantity":{
        "value": VQ_Value,
        "unit": VQ_units,
        "system": VQ_system,
        "code": VQ_units
    },
    "performer":[
            {
                "reference": P_reference_value,
                "display": P_reference_display  
            } 
    ],
    "referenceRange":[
                      {
                          "low":{
                              "value": Low_value,
                              "unit": RR_unit,
                              "system": RR_system,
                              "code": RR_Code_value
                          },
                          "high":{
                              "value": High_value,
                              "unit": RR_unit,
                              "system": RR_system,
                              "code": RR_Code_value
                          },
                      }
    ]
}
obv = Observation.parse_obj(Observation_String)
print(Observation_String)

{'resourceType': 'Observation', 'id': '10458131554-5', 'identifier': [{'use': 'official', 'value': '10458131554-5'}], 'status': 'final', 'code': {'coding': [{'system': 'http://loinc.org', 'code': '1554-5', 'display': 'GLUCOSE POST 12H CFST:MCNC:PT:SER/PLAS:QN'}]}, 'subject': {'reference': 'Patient/555-44-4444', 'display': 'EVE E. EVERYWOMAN'}, 'interpretation': [{'coding': [{'system': 'http://terminology.hl7.org/ValueSet/v3-ObservationInterpretationNormality', 'code': 'H', 'display': 'High'}]}], 'valueQuantity': {'value': '182', 'unit': 'mg/dl', 'system': 'http://unitsofmeasure.org', 'code': 'mg/dl'}, 'performer': [{'reference': 'Practitioner/444-44-4444', 'display': 'HOWARD H. HIPPOCRATES'}], 'referenceRange': [{'low': {'value': '70', 'unit': 'mg/dl', 'system': 'http://unitsofmeasure.org', 'code': 'mg/dl'}, 'high': {'value': '105', 'unit': 'mg/dl', 'system': 'http://unitsofmeasure.org', 'code': 'mg/dl'}}]}


In [ ]:
##Use fhir.resources to validate the Observation object
print(isinstance(obv, Observation))

True
